# Training Workflow

Notebook to bring everything together into a single training object.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from audiosep.model import convnet_model
from audiosep.data import load_data, save_mfcc, split_data
import joblib
import pickle

In [8]:
class Trainer(object):
    
    def __init__(self, json_path, **kwargs):
        self.json_path = json_path  #path of training mfccs
        self.kwargs = kwargs
        self.X_train, self.X_val, self.X_test, self.y_train, self.y_val, self.y_test = \
            split_data(dataset_path= self.json_path)
        self.input_shape = self.X_train.shape[1:]
        self.model = convnet_model(self.input_shape)
        self.genres = {0: "Blues",
                       1: "Classical",
                       2: "Country",
                       3: "Disco",
                       4: "Hiphop",
                       5: "Metal",
                       6: "pop",
                       7: "Reggae",
                       8: "Rock"
                      }
        
    def train(self, epochs= 30, batch_size= 32, verbose= 0):
        es = EarlyStopping(patience=10, restore_best_weights=True)

        self.model.fit(self.X_train, self.y_train,
                       epochs= epochs,
                       batch_size= batch_size,
                       validation_data= (self.X_val, self.y_val),
                       verbose= verbose,
                       callbacks= [es])
    def evaluate(self, test=False):
        if test:
            test_metrics = self.model.evaluate(self.X_test, self.y_test, verbose= 0)
            test_print = f"test accuracy: {test_metrics[1]}"
            print(test_print)
        train_metrics = self.model.evaluate(self.X_train, self.y_train, verbose= 0)
        val_metrics = self.model.evaluate(self.X_val, self.y_val, verbose= 0)
        
        train_print = f"train loss: {train_metrics[0]}, train accuracy: {train_metrics[1]}"
        val_print =  f"val loss: {val_metrics[0]}, val accuracy: {val_metrics[1]}"
        
        print(train_print)
        print(val_print)
        
    #def save_model(self):
    #    """Save the model into a joblib format"""
    #    joblib.dump(self.model, 'model.joblib')
    #    print(f"model.joblib saved locally.")
        
    def predict_new_song(self, X):
    
        test = save_mfcc(X, train= False, num_segments= 10, verbose=False)

        predictions = []

        for segment in test:
            segment = segment[np.newaxis, ...]
            seg_pred = self.model.predict(segment)
            seg_pred = np.argmax(seg_pred, axis=1)[0]
            predictions.append(seg_pred)

        predictions = np.array(predictions)

        values, counts = np.unique(predictions, return_counts= True)
        index = np.argmax(counts)
        
        # labels gotten from data.json['mapping']
        print(f"Predicted genre: {self.genres.get(values[index])}")
        
        #return values[index]
        

In [9]:
train = Trainer('../raw_data/genre/data.json')
train.train(epochs= 5, verbose= 2)
train.evaluate(test= True)

Epoch 1/5
85/85 - 2s - loss: 1.9116 - accuracy: 0.3545 - val_loss: 2.8334 - val_accuracy: 0.1289
Epoch 2/5
85/85 - 2s - loss: 1.3491 - accuracy: 0.5109 - val_loss: 1.3523 - val_accuracy: 0.5007
Epoch 3/5
85/85 - 2s - loss: 1.1499 - accuracy: 0.5858 - val_loss: 1.0522 - val_accuracy: 0.6104
Epoch 4/5
85/85 - 2s - loss: 1.0321 - accuracy: 0.6326 - val_loss: 0.9232 - val_accuracy: 0.6652
Epoch 5/5
85/85 - 2s - loss: 0.9361 - accuracy: 0.6681 - val_loss: 0.9026 - val_accuracy: 0.6874
test accuracy: 0.6737777590751648
train loss: 0.7256662845611572, train accuracy: 0.7549128532409668
val loss: 0.9025896191596985, val accuracy: 0.6874074339866638


In [10]:
#train.save_model()

# Test Model on new data

Quick test on a new wavefile, not to evaluate performance, but rather to test functionality and re-usability of the Trainer object.

Taken from local path. The same test track could be found [here](https://soundcloud.com/therealkulprit/the-city-is-mine-1).

In [11]:
path = "C:/Users/cezea/Desktop/MUSIC/exports/fela_vox.wav"

In [12]:
train.predict_new_song(path)

Predicted genre: Hiphop


In [ ]:
from audiosep.train

Correctly predicted.